In [2]:
import psycopg2
import pandas as pd
#import geopandas as gpd
import datetime
import shapely
import os

In [3]:
#Open images metadata from Digital Globe (see Jupyter Notebook)
digital_globe = pd.read_csv('/mnt/data/shared/imgs_metadata_2017.csv')
cols = ['catalogID', 'timestamp', 'browseURL', 'footprintWkt']
images = digital_globe[cols]
images.assign(timestamp = images['timestamp'].values.astype('datetime64[s]'))

,catalogID,timestamp,browseURL,footprintWkt
0,10200100619BAF00,2017-05-09 04:04:42,https://geobigdata.io/thumbnails/v1/browse/102...,"MULTIPOLYGON(((142.1453524 -10.5029861, 142.36..."
1,10200100627FC700,2017-05-09 04:04:52,https://geobigdata.io/thumbnails/v1/browse/102...,"MULTIPOLYGON(((142.1531241 -10.80287813, 142.3..."
2,10200100620D4C00,2017-06-04 03:53:18,https://geobigdata.io/thumbnails/v1/browse/102...,"MULTIPOLYGON(((142.2644451 -10.80582383, 142.4..."
3,103001006A74ED00,2017-06-04 00:53:18,https://geobigdata.io/thumbnails/v1/browse/103...,"MULTIPOLYGON(((142.5044171 -9.294755032, 142.6..."
4,104001002E078E00,2017-06-07 01:11:54,https://geobigdata.io/thumbnails/v1/browse/104...,"MULTIPOLYGON(((142.3253557 -10.82007855, 142.4..."
5,104A01002D807500,2017-06-07 01:12:09,https://geobigdata.io/thumbnails/v1/browse/104...,"MULTIPOLYGON(((142.2778162 -10.87718206, 142.3..."
6,104001002D2D1200,2017-06-07 01:12:49,https://geobigdata.io/thumbnails/v1/browse/104...,"MULTIPOLYGON(((142.1995686 -10.19949122, 142.3..."
7,104A01002E123C00,2017-06-07 01:11:37,https://geobigdata.io/thumbnails/v1/browse/104...,"MULTIPOLYGON(((142.2145778 -10.13984084, 142.3..."
8,104A01002801A800,2017-01-28 01:17:18,https://geobigdata.io/thumbnails/v1/browse/104...,"MULTIPOLYGON(((142.2979277 -10.86376932, 142.3..."
9,1030010063298200,2017-02-10 01:02:11,https://geobigdata.io/thumbnails/v1/browse/103...,"MULTIPOLYGON(((141.5406207 -8.979305446, 141.7..."


In [ ]:
from configparser import ConfigParser


def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)
 
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db


def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    
    try:
        params = config()
 
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
 
        cur = conn.cursor()
        cur.execute("select * from ais_messages.spire_ais where timestamp_ais between '2017-01-01 03:45:29' and '2017-07-19 01:12:49'")
        rows = cur.fetchall()
        spire_ais = pd.DataFrame(rows)
        return spire_ais

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 
 
if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...


In [9]:
spire_ais = connect()
spire_ais.columns = ['msg_type', 'mmsi','timestamp', 'nmea', 'status', 'rot', 'speed', 'accuracy', 'longitude', 'latitude', 'course', 'heading', 'manever']
print(spire_ais)

#intersection = pd.merge(spire_ais, images, how='inner', on=['timestamp'])
#print intersection

Connecting to the PostgreSQL database...
Database connection closed.
   msg_type       mmsi           timestamp  \
0         1  310384000 2017-06-06 09:03:29   
1         3  273847520 2017-05-09 23:44:16   
2         1  616708000 2017-05-14 21:00:42   
3         1  636016327 2017-05-03 11:53:10   
4         1  229443000 2017-05-30 12:57:53   

                                                nmea  status  rot  speed  \
0  \c:1496739809*5F\!AIVDM,1,1,,B,14`0EP002knaDd@...       0    0   17.9   
1  \c:1494373456*51\!AIVDM,1,1,,A,345:Dp?PA>;Ew7d...      15  720    7.8   
2  \c:1494795642*5A\!AIVDM,1,1,,A,19<8o`?010NjPKF...      15    0    6.4   
3  \c:1493812390*57\!AIVDM,1,1,,B,19NSFihOjg;K359...       0  720   17.5   
4  \c:1496149073*5B\!AIVDM,1,1,,B,13Jl5f00218VirQ...       0    0   12.9   

   accuracy   longitude   latitude  course  heading  manever  
0         1 -130.782920  53.434685   162.2      160      0.0  
1         0  158.594117  52.486950    22.2       27      0.0  
2       

(5, 13)